### **Import Libraries**

In [1]:
import cv2
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Loading files from tensorflow

url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = keras.utils.get_file('flower_photos', origin=url, cache_dir='.', untar=True)

In [3]:
# Import PathLib to convert the path to the windows type
import pathlib

data_dir = pathlib.Path(data_dir)
data_dir

WindowsPath('datasets/flower_photos')

In [5]:
# Analyzing the number of photos
img_num = len(list(data_dir.glob('*/*.jpg')))

print(f"We've a total of {img_num} images in this dataset")

We've a total of 3670 images in this dataset
